<a href="https://colab.research.google.com/github/koliby777/pokus-cislo/blob/master/uceni_modelu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
poznamky:
code explainer

https://monica.im/share/chat?shareId=zZqqbwEHGwKJStgy

"""

'\npoznamky:\ncode explainer\n\nhttps://monica.im/share/chat?shareId=zZqqbwEHGwKJStgy\n\n'

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F


batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

!wget https://raw.githubusercontent.com/koliby777/pokus-cislo/master/KYTICE/10x%20KYTICE/10x%20kytice.txt
with open('10x kytice.txt', 'r', encoding='utf-8') as f:
    text = f.read()


chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):


    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):


    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):


    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):


    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

print("----------------------------------------------------")
print("Model obsahuje ", sum(p.numel() for p in m.parameters())/1e6, 'miliónů parametrů celkem!')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


# učení modelu:

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")


    xb, yb = get_batch('train')


    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("naučeno !!!!!")

"""
# načtení parametrů modelu:
m = torch.load('C:/Users/sms88/Desktop/m.pth')  ?????
"""

# uložení parametrů modelu:
# torch.save(m, 'C:/Users/sms88/Desktop/model kytice.pth')


--2024-04-06 11:55:34--  https://raw.githubusercontent.com/koliby777/pokus-cislo/master/KYTICE/10x%20KYTICE/10x%20kytice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 806980 (788K) [text/plain]
Saving to: ‘10x kytice.txt’

10x kytice.txt      100%[===================>] 788.07K  --.-KB/s    in 0.04s   

2024-04-06 11:55:35 (17.2 MB/s) - ‘10x kytice.txt’ saved [806980/806980]

----------------------------------------------------
Model obsahuje  0.211148 miliónů parametrů celkem!
step 0: train loss 4.5069, val loss 4.5103
step 100: train loss 2.8025, val loss 2.8043
step 200: train loss 2.6582, val loss 2.6521
step 300: train loss 2.5969, val loss 2.5890
step 400: train loss 2.5523, val loss 2.5516
step 500: train loss 2.5122, val loss 2.5070
step 600: tr

"\n# načtení parametrů modelu:\nm = torch.load('C:/Users/sms88/Desktop/m.pth')  ?????\n"

In [4]:
# používání modelu:
print("----------------------------------------------------")
print('Zadej délku odpovědi modelu:')
max = int(input())
print("----------------------------------------------------")
print("Pro skončení zadej 'KONEC'")

def dotaz():
    print("----------------------------------------------------")
    print('Zadej dotaz:')
    text = input()
    return text

initial_text = dotaz()
while initial_text != "KONEC":
    initial_tokens = encode(initial_text)
    context_tensor = torch.tensor([initial_tokens], dtype=torch.long).to(device)  # Přidáváme dimenzi pro dávku
    generated_tokens = m.generate(context_tensor, max_new_tokens=max)  # Generuje zadaný počet tokenů
    generated_text = decode(generated_tokens[0, -max:].tolist())
    print("----------------------------------------------------")
    print("Odpověď:")
    print(generated_text)
    initial_text = dotaz()


----------------------------------------------------
Zadej délku odpovědi modelu:
200
----------------------------------------------------
Pro skončení zadej 'KONEC'
----------------------------------------------------
Zadej dotaz:
svatební košile
----------------------------------------------------
Odpověď:
,
můje protaz že, toříku bese až?

„Ždujem ože, kotý, sram dluhye, šame,
jak tole předí sečaté kokrovéma! Prolu, tyok cála,
a meje, hosp! kotlas - děje,
přejí,iv nadchoje jedritava jaké -
srav v jati 
----------------------------------------------------
Zadej dotaz:
svatební košile
----------------------------------------------------
Odpověď:
 vrder, dych hobý,
přadrajm káde, větá,
hozile méra jvomá pěl. -

Pátu svam deva tez mly mi čebu.
Nem jdce mečil víti ku,

o stru polo návše panně běké!

SPravoč mětš stomý, žik sely
astšíl již - a mi
----------------------------------------------------
Zadej dotaz:
KONEC
